# importing all libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from keras.preprocessing.sequence import pad_sequences
#import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [2]:
from tqdm import tqdm

In [3]:
import time

# creating path variables

In [4]:
path_all_datasets = "D:\\imp docs\\sem4\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\program files"
path_annotation_gzip ="D:\\imp docs\\sem4\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\annotations\\gzip annotations"

In [5]:
train_annot = "\\phoenix14t.pami0.train.annotations_only.gzip"
dev_annot = "\\phoenix14t.pami0.dev.annotations_only.gzip"
test_annot = "\\phoenix14t.pami0.test.annotations_only.gzip"

In [6]:
img_embedding = "\\emb_comp_i3d_train_zero.h5"
vocab_unique = "\\unique_vocab.pickle"
padded_gloss = "\\padded_gloss.h5"
gloss_vector_dataframe = "gloss_vector_dataframe.gzip"

In [7]:
dev_img_embedding = "\\emb_comp_i3d_dev_zero.h5"
test_img_embedding = "\\emb_comp_i3d_test_zero.h5"

# loading annotations in a dataframes

In [8]:
with gzip.open(path_annotation_gzip+train_annot,'rb') as f:
    annotation_gloss = pickle.load(f)

In [9]:
with gzip.open(path_annotation_gzip+dev_annot,'rb') as f:
    annotation_dev = pickle.load(f)

In [10]:
with gzip.open(path_annotation_gzip+test_annot,'rb')as f :
    annotation_test = pickle.load(f)

In [11]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [12]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)



In [13]:
annotation_gloss_test = pd.DataFrame(annotation_test)

In [14]:
annotation_gloss.head()

,name,signer,gloss,text
0,train/11August_2010_Wednesday_tagesschau-1,Signer08,JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR,und nun die wettervorhersage für morgen donner...
1,train/11August_2010_Wednesday_tagesschau-4,Signer08,ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...,mancherorts regnet es auch länger und ergiebig...
2,train/11August_2010_Wednesday_tagesschau-5,Signer08,NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...,im nordwesten bleibt es heute nacht meist troc...
3,train/11August_2010_Wednesday_tagesschau-6,Signer08,TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...,auch am tag gibt es verbreitet zum teil kräfti...
4,train/11August_2010_Wednesday_tagesschau-7,Signer08,WOLKE LOCH SPEZIELL NORDWEST,größere wolkenlücken finden sich vor allem im ...


In [15]:
annotation_gloss_dev.head()

,name,signer,gloss,text
0,dev/11August_2010_Wednesday_tagesschau-2,Signer08,DRUCK TIEF KOMMEN,tiefer luftdruck bestimmt in den nächsten tage...
1,dev/11August_2010_Wednesday_tagesschau-3,Signer08,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...,das bedeutet viele wolken und immer wieder zum...
2,dev/11August_2010_Wednesday_tagesschau-8,Signer08,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...,meist weht nur ein schwacher wind aus untersch...
3,dev/25October_2010_Monday_tagesschau-22,Signer01,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...,am mittwoch hier und da nieselregen in der nor...
4,dev/05May_2011_Thursday_tagesschau-25,Signer08,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...,und nun die wettervorhersage für morgen freita...


In [16]:
annotation_gloss_test.head()

,name,signer,gloss,text
0,test/25October_2010_Monday_tagesschau-17,Signer01,REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KO...,regen und schnee lassen an den alpen in der na...
1,test/25October_2010_Monday_tagesschau-24,Signer01,DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE W...,am donnerstag regen in der nordhälfte in der s...
2,test/15December_2010_Wednesday_tagesschau-37,Signer05,KRAEFTIG AB MORGEN FRUEH MEISTENS SCHNEE SCHNE...,vom nordmeer zieht ein kräftiges tief heran un...
3,test/10March_2011_Thursday_heute-58,Signer01,WOCHENENDE SONNE SAMSTAG SCHOEN TEMPERATUR BIS...,sonnig geht es auch ins wochenende samstag ein...
4,test/14August_2009_Friday_tagesschau-62,Signer05,DEUTSCH LAND MORGEN HOCH DRUCK KOMMEN WOLKE AU...,deutschland liegt morgen unter hochdruckeinflu...


In [17]:
# we can use values attribute to directly create a list of all sentences of a particular column
#annotation_gloss_test["text"].values

# preprocessing the vocabulary

In [18]:
for i in range(len(annotation_gloss)):
    annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
    annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
    annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
    #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [19]:
for i in range(len(annotation_gloss_dev)):
    annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
    annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
    annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [20]:
for i in range(len(annotation_gloss_test)):
    annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
    annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
    annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

# creating a vocabulary of texts

In [21]:
vocabulary = []
for txt in annotation_gloss.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_dev.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_dev.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_test.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_test.gloss.values:
    vocabulary.extend(txt.split())
print('Vocabulary Size: %d' % len(set(vocabulary)))

Vocabulary Size: 3606


# reading images embedding for train test and dev

In [22]:
#reading images feature vectors
with h5py.File(path_all_datasets+img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_train=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [23]:
#reading images feature vectors
with h5py.File(path_all_datasets+dev_img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_dev=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [24]:
temp_path_dev = "D:\\imp docs\\sem4\\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\program files"

In [25]:
data_dev.shape

(519, 512)

In [26]:
#data_dev[0]

In [27]:
 #data_dev_temp[0]

In [28]:
#reading images feature vectors
with h5py.File(path_all_datasets+test_img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_test=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [29]:
data_train.shape

(7096, 512)

In [30]:
data_dev.shape

(519, 512)

In [31]:
data_test.shape

(642, 512)

# creating better data representation

In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
import tensorflow.python.ops.numpy_ops.np_config as np_config


In [34]:
np_config.enable_numpy_behavior()

In [35]:
#pip install --upgrade tensorflow-lattice

In [36]:
import tensorflow_lattice.layers as tfl

In [151]:
data_train2= data_train.copy()
data_dev2 = data_dev.copy()
data_test2 = data_test.copy()

In [152]:
data_train2.shape

(7096, 512)

In [153]:
class FeatureEmbedder(nn.Module):
    
    def __init__(self, d_feat, d_model):
        super(FeatureEmbedder, self).__init__()
        self.d_model = d_model
        self.embedder = nn.Linear(d_feat, d_model)
        
    def forward(self, x): # x - tokens (B, seq_len, d_feat)
        x = self.embedder(x)
        x = x * np.sqrt(self.d_model)
        
        return x # (B, seq_len, d_model)

In [154]:
data_train2 = torch.from_numpy(data_train2)

In [155]:
data_train2 = data_train2.to(torch.float)

In [156]:
data_train2 = data_train2.reshape(7096,512,1)

In [157]:
features = FeatureEmbedder(1,8)

In [158]:
temp_data = features(data_train2)

In [72]:
#temp_data = temp_data.reshape(7096,8,512)

In [159]:
temp_data.shape

torch.Size([7096, 512, 8])

In [74]:
temp_data[0]

tensor([[-1.8660,  0.2782,  2.0802,  ...,  0.2566, -0.3102, -1.3358],
        [-1.8484,  0.4157,  1.9328,  ...,  0.2675, -0.3414, -1.3683],
        [-1.8608,  0.3187,  2.0368,  ...,  0.2658, -0.3365, -1.3632],
        ...,
        [-1.8586,  0.3360,  2.0183,  ...,  0.2918, -0.4110, -1.4407],
        [-1.8467,  0.4293,  1.9183,  ...,  0.2831, -0.3863, -1.4149],
        [-1.8646,  0.2894,  2.0682,  ...,  0.2676, -0.3417, -1.3686]],
       grad_fn=<SelectBackward>)

# creating tokenizer dataset


In [75]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
    gloss_token.append(annotation_gloss.iloc[i,2])
    text_token.append(annotation_gloss.iloc[i,3])

In [76]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
    gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
    text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [77]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
    gloss_token_test.append(annotation_gloss_test.iloc[i,2])
    text_token_test.append(annotation_gloss_test.iloc[i,3])

In [51]:
text_token_dev[0]

'startseq tiefer luftdruck bestimmt in den nächsten tagen unser wetter endseq'

# creating tokenizer

In [52]:
def create_tokenizer(vocabulary):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3606,
                                                 oov_token="<unk>",
                                                 filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(vocabulary)
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    return tokenizer

In [53]:
tokenizer = create_tokenizer(vocabulary)

In [54]:
vocab_size = len(tokenizer.word_index)+1
print("vocalb size",vocab_size)

vocalb size 3436


In [55]:
tokenizer.index_word[1]

'<unk>'

In [56]:
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# tokenizing train,test,dev glosses

In [57]:
train_y = encode_sequences(tokenizer,35,gloss_token)
dev_y = encode_sequences(tokenizer,35,gloss_token_dev)
test_y = encode_sequences(tokenizer,35,gloss_token_test)

In [58]:
train_y[0]

array([  2,  72,  31,   9,  70, 262, 327,   3,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

# hyperparameters

In [160]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
num_steps = 7096 // BATCH_SIZE

In [161]:
train_temp = temp_data.detach().numpy()

In [162]:
data_train_temp = tf.cast(train_temp,dtype = tf.float32)

In [163]:
#data_train = tf.cast(data_train,dtype = tf.float32)

In [164]:
def map_func(img_name, cap):
   
   return img_name, cap



In [165]:
dataset = tf.data.Dataset.from_tensor_slices((data_train_temp, train_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [84]:
# dataset = tf.data.Dataset.from_tensor_slices((data_train, train_y))
# dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [166]:
for (batch,(img,tar)) in enumerate(dataset.as_numpy_iterator()):
    print(batch)
    print(img)
    print("********")
    print(tar)
    

0
[[[-0.82786345  1.0213586  -0.86697257 ... -1.6018152   2.5989306
   -1.1299013 ]
  [-0.87957245  1.1092776  -0.99212223 ... -1.494182    2.6243696
   -1.0036058 ]
  [-0.83557296  1.0344669  -0.8856318  ... -1.5857677   2.6027234
   -1.1110712 ]
  ...
  [-0.84401274  1.0488167  -0.9060582  ... -1.5682002   2.6068754
   -1.0904579 ]
  [-0.84348     1.0479109  -0.90476876 ... -1.569309    2.6066132
   -1.0917591 ]
  [-0.8511991   1.0610355  -0.9234512  ... -1.5532415   2.610411
   -1.0729055 ]]

 [[-0.82614726  1.0184406  -0.8628189  ... -1.6053876   2.598086
   -1.134093  ]
  [-0.8856078   1.1195394  -1.0067294  ... -1.4816191   2.6273386
   -0.98886496]
  [-0.84153575  1.0446051  -0.90006316 ... -1.5733562   2.6056569
   -1.0965078 ]
  ...
  [-0.8573081   1.0714225  -0.9382367  ... -1.5405254   2.6134162
   -1.0579847 ]
  [-0.840899    1.0435225  -0.8985221  ... -1.5746814   2.6053436
   -1.098063  ]
  [-0.84840465  1.0562842  -0.9166878  ... -1.5590583   2.609036
   -1.0797309 ]]

 

   -1.0749711 ]]]
********
[[   2   15  197 ...    0    0    0]
 [   2   17  170 ...    0    0    0]
 [   2  147   74 ...    0    0    0]
 ...
 [   2   86  319 ...    0    0    0]
 [   2   33   88 ...    0    0    0]
 [   2   35 1843 ...    0    0    0]]
18
[[[-0.82964766  1.0243922  -0.8712908  ... -1.5981014   2.5998082
   -1.1255435 ]
  [-0.87568116  1.1026616  -0.9827045  ... -1.5022815   2.6224554
   -1.0131098 ]
  [-0.84237385  1.0460303  -0.90209174 ... -1.5716114   2.606069
   -1.0944606 ]
  ...
  [-0.8445847   1.0497892  -0.90744245 ... -1.5670097   2.6071568
   -1.0890609 ]
  [-0.84554785  1.0514269  -0.9097737  ... -1.5650047   2.6076307
   -1.0867083 ]
  [-0.8453458   1.0510833  -0.9092847  ... -1.5654253   2.6075313
   -1.0872018 ]]

 [[-0.83168966  1.0278641  -0.87623286 ... -1.593851    2.600813
   -1.1205562 ]
  [-0.8807366   1.1112571  -0.9949399  ... -1.4917587   2.6249423
   -1.0007623 ]
  [-0.8380379   1.0386578  -0.89159733 ... -1.580637    2.603936
   -1.105051  ]

[[  2 346  61 ...   0   0   0]
 [  2 136  18 ...   0   0   0]
 [  2 590  14 ...   0   0   0]
 ...
 [  2 176  41 ...   0   0   0]
 [  2 281 205 ...   0   0   0]
 [  2  23  74 ...   0   0   0]]
34
[[[-0.826338    1.018765   -0.8632806  ... -1.6049904   2.59818
   -1.133627  ]
  [-0.87529606  1.1020067  -0.98177236 ... -1.5030832   2.6222658
   -1.0140505 ]
  [-0.8519578   1.0623256  -0.9252874  ... -1.5516623   2.610784
   -1.0710524 ]
  ...
  [-0.84146905  1.0444919  -0.89990187 ... -1.5734948   2.605624
   -1.0966705 ]
  [-0.8481797   1.0559016  -0.91614336 ... -1.5595266   2.6089253
   -1.0802803 ]
  [-0.8361045   1.0353706  -0.8869181  ... -1.5846612   2.6029847
   -1.1097732 ]]

 [[-0.83034194  1.0255728  -0.8729711  ... -1.5966563   2.6001499
   -1.1238478 ]
  [-0.87947863  1.1091181  -0.9918952  ... -1.4943771   2.6243234
   -1.003835  ]
  [-0.83833504  1.0391631  -0.8923165  ... -1.5800184   2.604082
   -1.1043252 ]
  ...
  [-0.84845775  1.0563745  -0.9168164  ... -1.5589477   2.

[[[-0.8259069   1.018032   -0.86223704 ... -1.605888    2.5979679
   -1.1346803 ]
  [-0.88275707  1.1146923  -0.9998299  ... -1.4875531   2.6259363
   -0.9958277 ]
  [-0.8394069   1.0409856  -0.8949109  ... -1.5777873   2.6046095
   -1.1017072 ]
  ...
  [-0.8542006   1.0661389  -0.9307157  ... -1.5469939   2.6118875
   -1.0655745 ]
  [-0.8434477   1.047856   -0.9046906  ... -1.5693763   2.6065974
   -1.0918379 ]
  [-0.8555698   1.0684668  -0.93402934 ... -1.544144    2.612561
   -1.0622306 ]]

 [[-0.824678    1.0159426  -0.85926294 ... -1.6084458   2.5973635
   -1.1376815 ]
  [-0.88366145  1.1162301  -1.0020189  ... -1.4856704   2.6263814
   -0.9936186 ]
  [-0.8414496   1.0444586  -0.8998546  ... -1.5735356   2.6056144
   -1.0967182 ]
  ...
  [-0.8494996   1.0581458  -0.91933775 ... -1.5567793   2.6095748
   -1.0770568 ]
  [-0.847334    1.0544639  -0.91409665 ... -1.5612868   2.6085093
   -1.0823458 ]
  [-0.8529032   1.0639329  -0.9275754  ... -1.5496945   2.6112492
   -1.0687436 ]]

 

   -1.0799693 ]]]
********
[[  2  20  22 ...   0   0   0]
 [  2 176 126 ...   0   0   0]
 [  2  61  30 ...   0   0   0]
 ...
 [  2  17  79 ...   0   0   0]
 [  2 213  74 ...   0   0   0]
 [  2  70  23 ...   0   0   0]]
69
[[[-0.8272437   1.0203049  -0.86547256 ... -1.6031053   2.5986257
   -1.131415  ]
  [-0.8736327   1.0991787  -0.97774667 ... -1.5065454   2.6214476
   -1.0181131 ]
  [-0.85192496  1.0622697  -0.9252079  ... -1.5517308   2.6107678
   -1.0711328 ]
  ...
  [-0.84884745  1.0570371  -0.91775966 ... -1.5581365   2.6092541
   -1.0786493 ]
  [-0.84749335  1.0547348  -0.91448236 ... -1.560955    2.6085877
   -1.0819565 ]
  [-0.84438825  1.0494553  -0.90696704 ... -1.5674185   2.6070602
   -1.0895406 ]]

 [[-0.8251475   1.0167409  -0.8603993  ... -1.6074685   2.5975943
   -1.1365347 ]
  [-0.880992    1.1116914  -0.995558   ... -1.4912269   2.625068
   -1.0001385 ]
  [-0.8406387   1.04308    -0.89789206 ... -1.5752234   2.6052155
   -1.0986987 ]
  ...
  [-0.8500041   1.0590036  

[[[-0.822111    1.011578   -0.8530502  ... -1.613789    2.5961006
   -1.1439512 ]
  [-0.8808975   1.1115308  -0.99532944 ... -1.4914237   2.6250215
   -1.0003693 ]
  [-0.83992195  1.0418612  -0.8961573  ... -1.5767152   2.604863
   -1.1004493 ]
  ...
  [-0.8532693   1.0645552  -0.92846143 ... -1.5489326   2.6114292
   -1.0678495 ]
  [-0.8452987   1.0510033  -0.9091706  ... -1.5655234   2.6075082
   -1.0873169 ]
  [-0.8526557   1.063512   -0.92697644 ... -1.5502098   2.6111274
   -1.0693481 ]]

 [[-0.8237232   1.0143191  -0.85695195 ... -1.6104332   2.5968938
   -1.1400136 ]
  [-0.8816303   1.1127766  -0.9971028  ... -1.4898984   2.625382
   -0.9985797 ]
  [-0.84560585  1.0515255  -0.90991396 ... -1.5648842   2.607659
   -1.0865668 ]
  ...
  [-0.8509528   1.0606166  -0.92285496 ... -1.5537543   2.6102896
   -1.0735073 ]
  [-0.84076554  1.0432956  -0.89819914 ... -1.5749593   2.6052778
   -1.0983888 ]
  [-0.8493547   1.0578995  -0.91898715 ... -1.5570807   2.6095035
   -1.0774105 ]]

 [[

# model designinig


In [298]:
num_layer = 4
d_model = 512
dff = 512
num_heads = 8
row_size = 4
col_size = 512

target_vocab_size = len(tokenizer.word_index) + 1
dropout_rate = 0.01

In [170]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    #print("get angel")
    #print("angel shape",angle_rates.shape)
    return pos * angle_rates

In [126]:
# # new positional encoding for features
# def positional_encoding_2d(row,col,d_model):
#     drop_out =tf.keras.layers.Dropout(0.5)
#     pos_enc_mat = np.zeros((row,col))
#     odds = np.arange(1, d_model, 2)
#     evens = np.arange(0, d_model, 2)
    
#     for pos in range(rows):
#             pos_enc_mat[pos, even] = np.sin(pos / (10000 ** (odds / d_model)))
#             pos_enc_mat[pos, odd] = np.cos(pos / (10000 ** (evens / d_model)))
    
    
    
    
    
    
    
    
    
    
    
#     assert d_model % 2 == 0
#     row_pos = np.repeat(np.arange(row),col)[:,np.newaxis]
#     col_pos = np.repeat(np.expand_dims(np.arange(col),0),row,axis=0).reshape(-1,1)

#     angle_rads_row = get_angles(row_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
#     angle_rads_col = get_angles(col_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)

#     angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
#     angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
#     angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
#     angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
#     pos_encoding = np.concatenate([angle_rads_row,angle_rads_col],axis=1)[np.newaxis, ...]
#     return tf.cast(pos_encoding, dtype=tf.float32)


In [275]:
# def positional_encoding_2d(cols, dout_p=dropout_rate, seq_len=row_size): # 3651 max feat len for c3d
        
#         d_model = cols
#         print("d model shape",d_model)
#         dropout = tf.keras.layers.Dropout(dout_p)
        
#         pos_enc_mat = np.zeros((seq_len, d_model))
#         odds = np.arange(0, d_model, 2)
#         evens = np.arange(1, d_model, 2)
#         print("pos enc matrix shape",pos_enc_mat.shape)

#         for pos in range(seq_len):
#             pos_enc_mat[pos, odds] = np.sin(pos / (10000 ** (odds / d_model)))
#             pos_enc_mat[pos, evens] = np.cos(pos / (10000 ** (evens / d_model)))
        
#         #self.pos_enc_mat = torch.from_numpy(pos_enc_mat).unsqueeze(0)
#         pos_enc_mat = pos_enc_mat[np.newaxis,...]
#         #pos_enc_mat = pos_enc_mat.reshape(1,4096,2)
#         return tf.cast(pos_enc_mat,dtype = tf.float32)

In [276]:
def positional_encoding_2d(row,col,d_model):
    assert d_model % 2 == 0
    row_pos = np.repeat(np.arange(row),col)[:,np.newaxis]
    col_pos = np.repeat(np.expand_dims(np.arange(col),0),row,axis=0).reshape(-1,1)

    angle_rads_row = get_angles(row_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
    angle_rads_col = get_angles(col_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
    
#     angle_rads_col[0::2,:] = np.sin(angle_rads_row[0::2,:])
#     angle_rads_col[1::2,:] = np.cos(angle_rads_col[1::2,:])
#     pos_encoding = angle_rads_col[np.newaxis,...]

    angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
    angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
    angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
    angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
    pos_encoding = np.concatenate([angle_rads_row,angle_rads_col],axis=1)[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [277]:
def positional_encoding_1d(position, d_model):
    #print("positional encoding 1 dim")
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [278]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [279]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [280]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  #adding -Inf where mask is 1 s.t. value get ignored in softmax

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [281]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        #print("multihead attention")
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [282]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [283]:
# class ResidualConnection(tf.keras.layers.Layer):
    
#     def __init__(self, dout_p):
#         super(ResidualConnection, self).__init__()
#         self.norm = tf.keras.layers.LayerNormalization(0)
#         self.dropout = tf.keras.layers.Dropout(dout_p)
        
#     def call(self, x): # [(B, seq_len, d_model), attention or feed forward]
#         res = norm(x)
#         #res = sublayer(res)
#         res = self.dropout(res)
        
#         return x + res

In [284]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.01):
        super(EncoderLayer, self).__init__()
        

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        #print("encoder layer chota")
        

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [285]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.01):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, 
           look_ahead_mask=None, padding_mask=None):
        #print("decoder layer")
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        # using look ahead mask so that during self attention current query dont consider future token
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        # use padding mask to avoid padded values of both enc_output and dec_input
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

In [286]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
               row_size,col_size,rate=0.01):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Dense(self.d_model,activation='relu')
        #self.fnc = FeatureEmbedder(d_model,512)

        
        self.pos_encoding = positional_encoding_2d(row_size,col_size, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        # shape(x) = (batch_size,seq_len(H*W),features)
        #x = self.fnc(x)
        #print("encoder layer")
        #print("shape of x is ",x.shape)
        seq_len = tf.shape(x)[1]
        #print(tf.shape(x)[1])

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len(H*W), d_model)
        x += self.pos_encoding[:, :seq_len]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [287]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers,d_model,num_heads,dff, target_vocab_size, maximum_position_encoding,   rate=0.01):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding_1d(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,look_ahead_mask=None, padding_mask=None):
        seq_len = tf.shape(x)[1]
        attention_weights = {}
        #print("shape of seq_len ",seq_len)
        #print("x shape in decoder cakk",x.shape)

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                            look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights

In [288]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,row_size,col_size,
              target_vocab_size,max_pos_encoding, rate=0.01):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,row_size,col_size, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                          target_vocab_size,max_pos_encoding, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training,look_ahead_mask=None,dec_padding_mask=None,enc_padding_mask=None ):
        # print("inp.shape",inp.shape)
        # print("training shape",training.shape)
        # print("enc_padding_mask ",enc_padding_mask.shape)
        enc_output = self.encoder(inp, training)  # (batch_size, inp_seq_len, d_model      )
        dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        return final_output, attention_weights

In [289]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        #print("custom schedule")
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        #print("learning rate is",tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2))

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [290]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999, 
                                     epsilon=1e-9)

In [291]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [292]:

def loss_function(real, pred):
    #print("loss_function")
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [293]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [294]:
transformer = Transformer(num_layer,d_model,num_heads,dff,row_size,col_size,target_vocab_size,max_pos_encoding=target_vocab_size,rate=dropout_rate)

In [295]:

def create_masks_decoder(tar):
    #print("msak decoder run")
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    return combined_mask

In [296]:
@tf.function
def train_step(img_tensor, tar):
   #print("train run")
    tar_inp = tar[:, :-1]
    #print(tar_inp)

    tar_real = tar[:, 1:]
    # print(tar_real)
    dec_mask = create_masks_decoder(tar_inp)
    #print("j9j")
    with tf.GradientTape() as tape:
        predictions, _ = transformer(img_tensor, tar_inp,True, dec_mask)
#         print("actual value",tar_real)
#         print("predicted value",predictions)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)   
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [297]:
for epoch in range(30):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
  
    for (batch, (img_tensor, tar)) in enumerate(dataset.as_numpy_iterator()):
        #print("eopch run")
        train_step(img_tensor, tar)

        if batch % 50 == 0:
            

            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result()))


    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[64,8,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node transformer_9/encoder_9/encoder_layer_18/multi_head_attention_54/Softmax (defined at <ipython-input-280-d0ceb2ef3187>:31) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/transformer_9/decoder_5/embedding_5/embedding_lookup/Reshape/_28]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[64,8,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node transformer_9/encoder_9/encoder_layer_18/multi_head_attention_54/Softmax (defined at <i

# creating evaluation data

In [150]:
data_dev.shape

(519, 512)

In [156]:
# data_train2= data_t.copy()
data_dev2 = data_dev.copy()
# data_test2 = data_test.copy()

In [157]:
data_dev2 = torch.from_numpy(data_dev2)

In [158]:
data_dev2 = data_dev2.to(torch.float)

In [159]:
data_dev2 = data_dev2.reshape(519,512,1)

In [160]:
features_dev = FeatureEmbedder(1,2)

In [161]:
temp_data_dev = features_dev(data_dev2)

In [162]:
BATCH_SIZE = 1
BUFFER_SIZE = 1000
num_steps = 519 // BATCH_SIZE

In [163]:
temp_dev = temp_data_dev.detach().numpy()

In [164]:
data_dev_temp = tf.cast(temp_dev,dtype = tf.float32)

In [165]:
data_dev_temp.shape

TensorShape([519, 512, 2])

In [166]:
data_dev_temp = data_dev_temp.reshape(519,2,512)

In [167]:
dataset = tf.data.Dataset.from_tensor_slices((data_dev_temp, dev_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [168]:
for (batch,(img,tar)) in enumerate(dataset.as_numpy_iterator()):
    print(batch)
    print(img)
    print("********")
    print(tar)

0
[[[ 0.49902672 -0.7499879   0.52367467 ... -0.75707936  0.5088699
   -0.7601761 ]
  [ 0.5142492  -0.76574415  0.51299506 ... -0.7650069   0.50959665
   -0.7609284 ]]]
********
[[   2  107  425  112   53 1345    3    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0]]
1
[[[ 0.49765843 -0.7485716   0.52231735 ... -0.75783706  0.5074674
   -0.7587245 ]
  [ 0.51190984 -0.7633227   0.5126163  ... -0.7682232   0.5155089
   -0.767048  ]]]
********
[[  2  19  15  34  24  23 113   3   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
2
[[[ 0.4964518  -0.7473227   0.52551425 ... -0.7596307   0.51138747
   -0.7627821 ]
  [ 0.5131276  -0.7645832   0.51617026 ... -0.76574236  0.5112705
   -0.762661  ]]]
********
[[  2 126  41 263 160   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

117
[[[ 0.49849868 -0.7494413   0.5272349  ... -0.76259977  0.51149094
   -0.76288915]
  [ 0.5126215  -0.76405936  0.51948357 ... -0.76473755  0.5097695
   -0.7611073 ]]]
********
[[  2 147  62  28  33  42  57  36  28   3   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
118
[[[ 0.4978143  -0.7487329   0.5224224  ... -0.75999194  0.51106024
   -0.76244336]
  [ 0.5134896  -0.76495785  0.5170538  ... -0.76099217  0.5148097
   -0.7663243 ]]]
********
[[  2  19  15 150 194  14 196   8   3   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
119
[[[ 0.497884   -0.7488051   0.52149326 ... -0.75799716  0.5098537
   -0.76119447]
  [ 0.5111247  -0.76251     0.5144634  ... -0.7638736   0.51250356
   -0.76393723]]]
********
[[  2   9 244  28 445   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
120
[[[ 0.49394554 -0.

237
[[[ 0.49862626 -0.74957335  0.52661115 ... -0.7573317   0.5085677
   -0.75986344]
  [ 0.51236963 -0.7637986   0.5119805  ... -0.7648852   0.5100136
   -0.76136   ]]]
********
[[  2 267  71  22   3   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
238
[[[ 0.4979431  -0.74886626  0.52662516 ... -0.764051    0.5068155
   -0.7580497 ]
  [ 0.50801516 -0.75929147  0.516606   ... -0.7659248   0.51456887
   -0.76607496]]]
********
[[  2  12 142  34 121 349  26  99   3   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
239
[[[ 0.49811256 -0.7490417   0.52591205 ... -0.7585295   0.50827307
   -0.75955844]
  [ 0.5135529  -0.76502335  0.5144502  ... -0.7639441   0.51286674
   -0.7643132 ]]]
********
[[   2 1466   20   17   93  156   54  146   23  190  134  190  127  165
     3    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   

   -0.7619028 ]]]
********
[[  2 263 160 126  41   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
360
[[[ 0.51597494 -0.7675303   0.48605755 ... -0.7638401   0.51739043
   -0.76899546]
  [ 0.51006526 -0.7614135   0.48364013 ... -0.75685304  0.50250506
   -0.75358814]]]
********
[[  2  19  15 149 122  17 156 117 257 134  54 170  54 119   3   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
361
[[[ 0.5008318  -0.75185627  0.52507716 ... -0.7594258   0.5102381
   -0.7615924 ]
  [ 0.5121662  -0.7635881   0.5121977  ... -0.76509774  0.51165956
   -0.7630637 ]]]
********
[[  2   9 345 207 254 579 147  36  28 128  69   3   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
362
[[[ 0.4972963  -0.74819684  0.5261397  ... -0.75922924  0.509639
   -0.7609723 ]
  [ 0.51376605 -0.76524395  0.51392305 ... -0.76650786  0.5138618
   -0.76534307]]]
*****

477
[[[ 0.49680677 -0.74769014  0.52738273 ... -0.75941616  0.509218
   -0.7605365 ]
  [ 0.5128959  -0.7643434   0.51469576 ... -0.76453424  0.5137443
   -0.76522154]]]
********
[[  2 126  41 263 160   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
478
[[[ 0.49317178 -0.7439277   0.52107537 ... -0.76078767  0.51072943
   -0.762101  ]
  [ 0.5065608  -0.75778615  0.5162472  ... -0.7696963   0.5142509
   -0.7657458 ]]]
********
[[  2  23  17  74 135  26   3   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
479
[[[ 0.4990094  -0.74996996  0.52785975 ... -0.7622235   0.50951153
   -0.76084036]
  [ 0.5137125  -0.7651886   0.51370937 ... -0.76525384  0.51113665
   -0.76252246]]]
********
[[  2 113 220 211  48  18   3   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
480
[[[ 0.49931514 -0.7

In [137]:
# @tf.function
# def train_step(img_tensor, tar):
#    #print("train run")
#     tar_inp = tar[:, :-1]
#     #print(tar_inp)

#     tar_real = tar[:, 1:]
#     # print(tar_real)
#     dec_mask = create_masks_decoder(tar_inp)
#     #print("j9j")
#     with tf.GradientTape() as tape:
#         predictions, _ = transformer(img_tensor, tar_inp,True, dec_mask)
#         loss = loss_function(tar_real, predictions)

#     gradients = tape.gradient(loss, transformer.trainable_variables)   
#     optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
#     train_loss(loss)
#     train_accuracy(tar_real, predictions)

In [138]:
pip install jiwer

  Using cached jiwer-2.2.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [139]:
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

In [140]:
import jiwer

In [169]:
def evaluate(image_tensor):

#     temp_input = tf.expand_dims(load_image(image)[0], 0)
#     img_tensor_val = image_features_extract_model(temp_input)
    #img_tensor_val = tf.reshape(img_tensor, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    start_token = tokenizer.word_index['startseq']
    end_token = tokenizer.word_index['endseq']

    #decoder input is start token.
    decoder_input = [start_token]
    output = tf.expand_dims(decoder_input, 0) #tokens
    result = [] #word list

    for i in range(35):
        dec_mask = create_masks_decoder(output)

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(img_tensor,output,False,dec_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        # return the result if the predicted_id is equal to the end token
        if predicted_id == end_token:
            return result,tf.squeeze(output, axis=0), attention_weights
        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        result.append(tokenizer.index_word[int(predicted_id)])
        output = tf.concat([output, predicted_id], axis=-1)

        return result,tf.squeeze(output, axis=0), attention_weights

In [174]:
actual,predicted = list(),list()
for (batch, (img_tensor, tar)) in enumerate(dataset.as_numpy_iterator()):
        #print("eopch run")
        #train_step(img_tensor, tar)
        #image = data_dev[i]
        caption,result,attention_weights = evaluate(img_tensor)
        real_caption = text_token_dev[batch].split(' ')
        for j in real_caption:
            if((j=='startseq')or (j=="endseq")):
                  real_caption.remove(j)
        first = ' '.join(j for j in real_caption)
        for j in caption:
            if j=="<unk>":
                  caption.remove(j)

        # for j in real_caption:
        #   if (j=="<unk>") or(j=="endseq") :
        #     real_caption.remove(j)
        #first = real_caption.split(' ', 1)[1]
        result_join = ' '.join(caption)
        result_final = result_join.rsplit(' ', 1)[0]
        print("predicted sentence",result_final)
        print("actual sentence",first)
        actual.append(first)
        predicted.append(result_final)
        # print('BLEU-1: %f' % corpus_bleu([first],[result_final], weights=(1.0, 0, 0, 0),smoothing_function=smoothie))
        # print('BLEU-2: %f' % corpus_bleu([first],[result_final], weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie))
        # print('BLEU-3: %f' % corpus_bleu([first],[result_final], weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie))
        # print('BLEU-4: %f' % corpus_bleu([first],[result_final], weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie))
        print("word error rate is",jiwer.wer(first,result_final))
    

print('BLEU-1: %f' % corpus_bleu(actual,predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie))
print('BLEU-2: %f' % corpus_bleu(actual,predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie))
print('BLEU-3: %f' % corpus_bleu(actual,predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie))
print('BLEU-4: %f' % corpus_bleu(actual,predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie))

predicted sentence morgen
actual sentence tiefer luftdruck bestimmt in den nächsten tagen unser wetter
word error rate is 1.0
predicted sentence morgen
actual sentence das bedeutet viele wolken und immer wieder zum teil kräftige schauer und gewitter
word error rate is 1.0
predicted sentence morgen
actual sentence meist weht nur ein schwacher wind aus unterschiedlichen richtungen der bei schauern und gewittern stark böig sein kann
word error rate is 1.0
predicted sentence morgen
actual sentence am mittwoch hier und da nieselregen in der nordwesthälfte an den küsten kräftiger wind
word error rate is 1.0
predicted sentence morgen
actual sentence und nun die wettervorhersage für morgen freitag den sechsten mai
word error rate is 0.9
predicted sentence morgen
actual sentence am tag breiten sich die teilweise kräftigen schneefälle weiter aus
word error rate is 1.0
predicted sentence morgen
actual sentence es wird wärmer aus dem südwesten morgen haben wir schon über frankreich die fünfzehn bi

predicted sentence morgen
actual sentence am samstag viel sonne ganz im norden einzelne schauer
word error rate is 1.0
predicted sentence morgen
actual sentence und nun die wettervorhersage für morgen freitag den dreißigsten oktober
word error rate is 0.9
predicted sentence morgen
actual sentence nachmittags kann hier und da die sonne durchaus mal rauskommen
word error rate is 1.0
predicted sentence morgen
actual sentence liebe zuschauer guten abend
word error rate is 1.0
predicted sentence morgen
actual sentence ein ausläufer des tiefs über dem nordmeer lenkt allerdings von nordwesten wolkenfelder ins land
word error rate is 1.0
predicted sentence morgen
actual sentence im westen könnte es hier westlich des rheins nachmittags ein bisschen freundlicher werden und die sonne durchaus mal rauskommen
word error rate is 1.0
predicted sentence morgen
actual sentence der ausläufer eines tiefs verlagert sich morgen vom westen deutschlands weiter nach osten
word error rate is 0.9230769230769231

predicted sentence morgen
actual sentence zur zeit gibt es aber noch teilweise unwetterartige gewitter der deutsche wetterdienst hat entsprechende unwetterwarnungen herausgegeben
word error rate is 1.0
predicted sentence morgen
actual sentence im süden bleibt es unter hochdruckeinfluss bei dem ruhigen und teilweise freundlichen herbstwetter
word error rate is 1.0
predicted sentence morgen
actual sentence ähnliches wetter auch am samstag von norden wird es allerdings kühler
word error rate is 1.0
predicted sentence morgen
actual sentence es weht meist ein schwacher wind aus nördlichen bis westlichen richtungen
word error rate is 1.0
predicted sentence morgen
actual sentence im emsland morgen sechs grad über null im allgäu frostige minus vier grad
word error rate is 0.9230769230769231
predicted sentence morgen
actual sentence morgen reichen die werte von achtzehn grad in der eifel bis sechsundzwanzig grad in der lausitz
word error rate is 0.9375
predicted sentence morgen
actual sentence 

predicted sentence morgen
actual sentence danach erwartet uns im ganzen land unbeständiges wetter mit weiteren schneefällen
word error rate is 1.0
predicted sentence morgen
actual sentence und nun die wettervorhersage für morgen dienstag den neunten februar
word error rate is 0.9
predicted sentence morgen
actual sentence die aussichten am samstag hier und der schnee vor allem im norden zeigt sich auch mal die sonne
word error rate is 1.0
predicted sentence morgen
actual sentence heute nacht temperaturen von minus fünf grad im nordosten bis plus drei grad am rhein
word error rate is 1.0
predicted sentence morgen
actual sentence dort morgen bis plus sechs grad in vorpommern gibt es leichten frost
word error rate is 0.9166666666666666
predicted sentence morgen
actual sentence deutlich milder ist es unter den wolken norddeutschlands
word error rate is 1.0
predicted sentence morgen
actual sentence aber tagsüber ist es dann dort kühler als in süddeutschland
word error rate is 1.0
predicted s

predicted sentence morgen
actual sentence in der lausitz heute nacht bis fünfzehn im sauerland nur kühle sechs grad
word error rate is 1.0
predicted sentence morgen
actual sentence auch in der neuen woche geht es meist sonnig und warm weiter
word error rate is 1.0
predicted sentence morgen
actual sentence in den nächsten tagen erwartet uns wechselhaftes wetter
word error rate is 1.0
predicted sentence morgen
actual sentence kühle luft die zwischen dem tief im norden und dem hoch im westen heranfließt breitet sich morgen in ganz deutschland aus
word error rate is 0.9523809523809523
predicted sentence morgen
actual sentence auch morgen beschert uns hoher luftdruck viel sonnenschein
word error rate is 0.875
predicted sentence morgen
actual sentence die aussichten am sonntag vor allem im osten noch schneefälle an den küsten bleibt es windig
word error rate is 1.0
predicted sentence morgen
actual sentence es wird auch langsam wärmer am rhein dann schon mal so achtzehn bis zwanzig grad im üb

...

predicted sentence morgen
actual sentence heute nacht ja schon regen im westen deutschlands der sich auch langsam richtung elbe ausbreitet
word error rate is 1.0
predicted sentence morgen
actual sentence klingt schon leicht herbstlich
word error rate is 1.0
predicted sentence morgen
actual sentence jetzt wünsche ich ihnen noch einen schönen abend
word error rate is 1.0
predicted sentence morgen
actual sentence im osten und südosten scheint morgen verbreitet die sonne in der nordwesthälfte dagegen viele wolken und stellenweise etwas regen richtung süden auch ein paar schauer
word error rate is 0.96
predicted sentence morgen
actual sentence und nun die wettervorhersage für morgen sonntag den vierzehnten februar
word error rate is 0.9
predicted sentence morgen
actual sentence die aussichten am montag teils sonne teils wolken im westen schneit es etwas
word error rate is 1.0
predicted sentence morgen
actual sentence denn es sieht fast so aus als ob das hintere tief das vordere tief einholt

predicted sentence morgen
actual sentence im südosten schneit es etwas sonst ist es trocken bei minus acht bis plus ein grad
word error rate is 1.0
predicted sentence morgen
actual sentence morgen regnet es verbreitet im bergland zum teil schnee wobei die schneefallgrenze im süden vorübergehend auf etwa zwölfhundert meter steigt
word error rate is 0.95
predicted sentence morgen
actual sentence am tag einundzwanzig grad auf sylt und neunundzwanzig grad in der lausitz und in niederbayern
word error rate is 1.0
predicted sentence morgen
actual sentence aber jetzt werden sie nach osten abziehen dabei die schwüle luft mitnehmen
word error rate is 1.0
predicted sentence morgen
actual sentence dahinter kommt zwar auch wieder etwas regen aber auch deutlich angenehmere luft
word error rate is 1.0
predicted sentence morgen
actual sentence heute nacht sowie ab morgen nachmittag an der küste und in den hochlagen von harz und erzgebirge schwere sturmböen
word error rate is 0.9473684210526315
predic

predicted sentence morgen
actual sentence in der nacht temperaturen von null bis fünf am main bis sieben grad
word error rate is 1.0
predicted sentence morgen
actual sentence und nun die wettervorhersage für morgen donnerstag den vierten oktober
word error rate is 0.9
predicted sentence morgen
actual sentence im südosten noch meist trocken
word error rate is 1.0
predicted sentence morgen
actual sentence und dort gibt es morgen zunächst größere wolkenlücken
word error rate is 0.875
predicted sentence morgen
actual sentence sonst regnet es teilweise kräftig
word error rate is 1.0
predicted sentence morgen
actual sentence abgesehen von schauer und gewitterböen weht der wind im norden schwach bis mäßig sonst schwach aus nordwest
word error rate is 1.0
predicted sentence morgen
actual sentence heute nacht temperaturen zwischen elf und achtzehn grad
word error rate is 1.0
predicted sentence morgen
actual sentence am tag zwei grad ganz im osten und dreizehn grad im nordwesten
word error rate 

predicted sentence morgen
actual sentence am freitag wieder ruhigeres wetter und kühler
word error rate is 1.0
predicted sentence morgen
actual sentence dieser kräftige regen fällt heute nacht von der ostsee über sachsen bis nach bayern auch sonst oft bedeckt in schleswig holstein sowie im südwesten auch wolkenlücken
word error rate is 1.0
predicted sentence morgen
actual sentence heute nacht dreizehn bis drei grad an den alpen werte um den gefrierpunkt
word error rate is 1.0
predicted sentence morgen
actual sentence da kommen die gewitter erst morgen nachmittag hin
word error rate is 0.875
predicted sentence morgen
actual sentence und morgen früh ist es mild in ganz deutschland acht bis sechzehn grad
word error rate is 0.9230769230769231
predicted sentence morgen
actual sentence aber im osten haben wir noch anfangs die wärme vierundzwanzig grad genauso wie im östlichen bayern
word error rate is 1.0
predicted sentence morgen
actual sentence dazu bläst ein böiger nordostwind an der osts

NameError: name 'corpus_bleu' is not defined

In [ ]:
start_token = tokenizer.word_index['<start>']
end_token = tokenizer.word_index['<end>']
# select random image from validation data
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
caption,result,attention_weights = evaluate(image)

first = real_caption.split(' ', 1)[1]
real_caption = first.rsplit(' ', 1)[0]

#remove "<unk>" in result
for i in caption:
    if i=="<unk>":
        caption.remove(i)

for i in real_caption:
    if i=="<unk>":
        real_caption.remove(i)

#remove <end> from result         
result_join = ' '.join(caption)
result_final = result_join.rsplit(' ', 1)[0]

real_appn = []
real_appn.append(real_caption.split())
reference = real_appn
candidate = caption
score = sentence_bleu(reference, candidate, weights=(1.0,0,0,0))
print(f"BLEU-1 score: {score*100}")
score = sentence_bleu(reference, candidate, weights=(0.5,0.5,0,0))
print(f"BLEU-2 score: {score*100}")
score = sentence_bleu(reference, candidate, weights=(0.3,0.3,0.3,0))
print(f"BLEU-3 score: {score*100}")
score = sentence_bleu(reference, candidate, weights=(0.25,0.25,0.25,0.25))
print(f"BLEU-4 score: {score*100}")

print ('Real Caption:', real_caption)
print ('Predicted Caption:', ' '.join(caption))
temp_image = np.array(Image.open(image))
plt.imshow(temp_image)